In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [2]:
df_dataset = pd.read_csv('/home/neil/scipy19/scipy19/assets/linregr/regr.csv')
df_dataset.dtypes
scaler = StandardScaler()
df_xy = pd.DataFrame(scaler.fit_transform(df_dataset), columns = list(df_dataset.columns))
X = df_xy.drop("Y house price of unit area", axis=1).values
Y = df_xy[["Y house price of unit area"]].values
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size = 0.2,
                                                    random_state=1)
model_reg = LinearRegression().fit(X_train, y_train)
model_reg.intercept_
model_reg.coef_
model_reg.score(X_train, y_train)
model_reg.score(X_test, y_test)
y_pred = model_reg.predict(X_test)

In [3]:
df_basket = pd.read_csv('/home/neil/scipy19/scipy19/assets/apriori/baskets.csv')
df_grp_basket = df_basket.groupby(['order_id'])['product_type'].apply('|'.join).reset_index()
orders = [prods.split('|') for _, prods in df_grp_basket.values]
te = TransactionEncoder()
te_ary = te.fit(orders).transform(orders)
df = pd.DataFrame(te_ary, columns = te.columns_)
itemsets = apriori(df, min_support=0.1, use_colnames=True).sort_values(by="support", ascending = False)

In [4]:
df_risk = pd.read_csv('/home/neil/scipy19/scipy19/assets/creditrisk/risk.csv')
df_risk = df_risk.replace('?', pd.NaT)
df_risk = df_risk.dropna()
df_risk.dtypes

Male               object
Age                object
Debt              float64
YearsEmployed     float64
PriorDefault       object
Employed           object
Credit Score        int64
DriversLicense     object
Income              int64
Approved           object
dtype: object

In [5]:
df_risk['Male'] = df_risk['Male'].astype('category')
df_risk['Age'] = df_risk['Age'].astype('category')
df_risk['PriorDefault'] = df_risk['PriorDefault'].astype('category')
df_risk['Employed'] = df_risk['Employed'].astype('category')
df_risk['DriversLicense'] = df_risk['DriversLicense'].astype('category')
df_risk['Approved'] = df_risk['Approved'].astype('category')

In [6]:
cat_cols = df_risk.select_dtypes(['category']).columns
df_xy = df_risk.copy()

In [7]:
df_xy[cat_cols] = df_xy[cat_cols].apply(lambda x: x.cat.codes)

In [8]:
X, y = df_xy.drop(["Approved"], axis=1).values, df_xy["Approved"].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state=1)
model = KNeighborsClassifier(n_neighbours = 3)
model.fit(X_train, y_train.ravel())
y_pred = model.predict(X_test)
model.score(X_test, y_pred)

TypeError: __init__() got an unexpected keyword argument 'n_neighbours'